In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [4]:
features_df = pd.read_csv('1112028701-5fSGPkcoJNxUoRraRcJ9WQ.csv')

In [5]:
features_df.head()

,id,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode,type,uri
0,6nz8W5q940pEthdQfjwYhh,0.213,0.101,134.968,0.0329,0.882000,0.851000,3,0.354,2,198444,-16.463,0.158,0,audio_features,spotify:track:6nz8W5q940pEthdQfjwYhh
1,7sBRwgYSf6dwyIi9T6uY7y,0.223,0.114,109.466,0.0651,0.970000,0.888000,4,0.758,8,213637,-13.275,0.265,1,audio_features,spotify:track:7sBRwgYSf6dwyIi9T6uY7y
2,4PYoRZpEoJHAi22nbeGA0b,0.844,0.238,119.987,0.0407,0.000186,0.539000,4,0.539,1,199800,-5.604,0.466,1,audio_features,spotify:track:4PYoRZpEoJHAi22nbeGA0b
3,3JCvbJlquC5ZhdXhHq62Z5,0.991,0.290,83.995,0.0960,0.000018,0.000000,4,0.477,5,192261,-2.692,0.364,0,audio_features,spotify:track:3JCvbJlquC5ZhdXhHq62Z5
4,3tyFkV691mLN162OMYP5Vi,0.659,0.118,106.913,0.0293,0.002110,0.000017,4,0.573,3,266947,-5.271,0.204,0,audio_features,spotify:track:3tyFkV691mLN162OMYP5Vi


In [6]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 16 columns):
id                  705 non-null object
energy              705 non-null float64
liveness            705 non-null float64
tempo               705 non-null float64
speechiness         705 non-null float64
acousticness        705 non-null float64
instrumentalness    705 non-null float64
time_signature      705 non-null int64
danceability        705 non-null float64
key                 705 non-null int64
duration_ms         705 non-null int64
loudness            705 non-null float64
valence             705 non-null float64
mode                705 non-null int64
type                705 non-null object
uri                 705 non-null object
dtypes: float64(9), int64(4), object(3)
memory usage: 88.2+ KB


In [7]:
features_df.describe()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode
count,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000,705.000000
mean,0.693043,0.184952,124.726033,0.064389,0.169982,0.166389,3.919149,0.548540,5.489362,268839.582979,-7.211206,0.454392,0.551773
std,0.218178,0.155091,26.948960,0.058233,0.269039,0.305278,0.325069,0.167452,3.475599,87209.176094,3.420787,0.227066,0.497665
min,0.056300,0.015600,68.992000,0.023200,0.000004,0.000000,1.000000,0.154000,0.000000,55973.000000,-24.738000,0.035600,0.000000
25%,0.546000,0.097000,105.821000,0.034200,0.001820,0.000002,4.000000,0.432000,2.000000,212310.000000,-8.966000,0.279000,0.000000
50%,0.748000,0.121000,123.781000,0.044800,0.019700,0.000572,4.000000,0.548000,6.000000,244720.000000,-6.485000,0.433000,1.000000
75%,0.873000,0.221000,140.049000,0.066000,0.223000,0.138000,4.000000,0.671000,9.000000,304560.000000,-4.626000,0.628000,1.000000
max,0.997000,0.991000,245.997000,0.497000,0.979000,0.950000,5.000000,0.928000,11.000000,759360.000000,-1.606000,0.966000,1.000000
